In [1]:
import time
import os

import requests
import pandas as pd

import pickle
from pymongo import MongoClient

from config import genius_access_token

### Connecting to MongoDB

In [2]:
client = MongoClient()

In [3]:
db = client.genius

In [4]:
db.list_collection_names()

['song_urls', 'song_lyrics', 'artist_ids']

## Get Artist IDs

In [5]:
cursor = db.artist_ids.find({}, {'_id':0, 'artist_id':1})
artist_ids = pd.DataFrame(list(cursor))
artist_ids = artist_ids.artist_id.values.tolist()
artist_ids[:5]

[358098, 63636, 1602422, 353058, 21765]

## Write to mongoDB

In [6]:
def get_json(headers, artist_id, page_num):
    url = "http://api.genius.com/artists/{}/songs?page={}".format(artist_id, page_num)
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    json = resp.json()
    return json

In [9]:
def save_song_info_to_mongoDB(json, artist_id):
    song_list = json['response']['songs']
    
    for song in song_list:
        song_info = {}
        song_info['artist_id'] = artist_id
        features = ['title','id','url']
        
        for feature in features:
            song_info[feature] = song[feature]
            
        db.song_urls.insert_one(song_info)

In [8]:
def get_artist_ids(artist_ids, access_token, headers):
    
    for artist_id in artist_ids:
        page_num = 1
        json = get_json(headers, artist_id, page_num)
        save_song_info_to_mongoDB(json, artist_id)
        
        while json['response']['next_page']:
            page_num += 1
            json = get_json(headers, artist_id, page_num)
            save_song_info_to_mongoDB(json, artist_id)

In [10]:
access_token = genius_access_token
headers={
         "Accept": "application/json",
         "Content-Type": "application/json",
         "Authorization": "Bearer " + access_token
        }

In [11]:
get_artist_ids(artist_ids, access_token, headers)

In [7]:
access_token = genius_access_token
headers={
         "Accept": "application/json",
         "Content-Type": "application/json",
         "Authorization": "Bearer " + access_token
        }

In [8]:

url = "http://api.genius.com/artists/{}/songs?page={}".format(artist_ids[0], 1)
resp = requests.get(url, headers=headers)
resp.raise_for_status()
json = resp.json()

In [13]:
json['response']['songs'][1]['url']

'https://genius.com/The-essex-green-big-green-tree-lyrics'